# *Traffic Sign Recognizer - 99% accuracy*

## Importing Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import torch
# from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(42)

from matplotlib import style
style.use('fivethirtyeight')

## Assigning Path for Dataset

In [ ]:
data_dir = './data'
train_path = './data/Train'
test_path = './data/Test'

# Resizing the images to 30x30x3
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3

## Finding Total Classes

In [ ]:
NUM_CATEGORIES = len(os.listdir(train_path))
NUM_CATEGORIES

In [ ]:
# Label Overview
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

## Visualizing The Dataset

In [ ]:
folders = os.listdir(train_path)

train_number = []
class_num = []

for folder in folders:
    train_files = os.listdir(train_path + '/' + folder)
    train_number.append(len(train_files))
    class_num.append(classes[int(folder)])
    
# Sorting the dataset on the basis of number of images in each class
zipped_lists = zip(train_number, class_num)
sorted_pairs = sorted(zipped_lists)
tuples = zip(*sorted_pairs)
train_number, class_num = [ list(tuple) for tuple in  tuples]

# Plotting the number of images in each class
plt.figure(figsize=(21,10))  
plt.bar(class_num, train_number)
plt.xticks(class_num, rotation='vertical')
plt.show()

In [ ]:
# Visualizing 25 random images from test data
import random
from matplotlib.image import imread

test = pd.read_csv(data_dir + '/Test.csv')
imgs = test["Path"].values

plt.figure(figsize=(25,25))

for i in range(1,26):
    plt.subplot(5,5,i)
    random_img_path = data_dir + '/' + random.choice(imgs)
    rand_img = imread(random_img_path)
    plt.imshow(rand_img)
    plt.grid(b=None)
    plt.xlabel(rand_img.shape[1], fontsize = 20)#width of image
    plt.ylabel(rand_img.shape[0], fontsize = 20)#height of image

## Collecting the Training Data

In [ ]:


image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)

    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
            resize_image = np.array(resize_image)
            image_data.append(resize_image)
            image_labels.append(i)
        except:
            print("Error in " + img)

# Changing the list to numpy array
image_data = np.array(image_data)
image_labels = np.array(image_labels)

print(image_data.shape, image_labels.shape)

## Shuffling the training data

In [ ]:
shuffle_indexes = np.arange(image_data.shape[0])
np.random.shuffle(shuffle_indexes)
image_data = image_data[shuffle_indexes]
image_labels = image_labels[shuffle_indexes]

## Splitting the data into train and validation set

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(image_data, image_labels, test_size=0.3, random_state=42, shuffle=True)

X_train = X_train/255 
X_val = X_val/255

print("X_train.shape", X_train.shape)
print("X_valid.shape", X_val.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_val.shape)

## One hot encoding the labels

In [ ]:
train_shape = y_train.shape
val_shape = y_val.shape
y_train = torch.tensor(y_train).unsqueeze_(1)
y_train = torch.zeros(train_shape[0], NUM_CATEGORIES).scatter_(1, y_train, 1)
y_val = torch.tensor(y_val).unsqueeze_(1)
y_val = torch.zeros(val_shape[0], NUM_CATEGORIES).scatter_(1, y_val, 1)

X_train = torch.tensor(X_train).permute(0,3,1,2)
X_val = torch.tensor(X_val).permute(0,3,1,2)

print(y_train.shape)
print(y_val.shape)
print(X_train.shape)


## Making the model

In [ ]:
from models.model import Model
model = Model(NUM_CATEGORIES).cuda()


In [ ]:
lr = 0.004
epochs = 30

optimizer = torch.optim.Adam(
    model.parameters(), lr=lr, weight_decay=lr/(epochs*0.5))
#  Adam(lr=lr, decay=lr / (epochs * 0.5))
criterion = torch.nn.CrossEntropyLoss().cuda()
lr_decayer = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 'min', factor=0.5, patience=2, verbose=True)
val_loss = float('Inf')


## Augmenting the data and training the model

In [ ]:
from models.data_loader import MyDataLoader
from models.data_loader import pca
train_folder = MyDataLoader(X_train, y_train)
valid_folder = MyDataLoader(X_val, y_val)

train_loader = torch.utils.data.DataLoader(
    train_folder,
    batch_size=32, shuffle=True,
    num_workers=16, pin_memory=True,
    drop_last=True
)

val_loader = torch.utils.data.DataLoader(
    valid_folder,
    batch_size=32, shuffle=False,
    num_workers=16, pin_memory=True,
    drop_last=False
)

model.double()

In [ ]:
for epoch in range(epochs):

    losses = 0.
    precs = 0.

    model.train()

    for i, (input, target) in enumerate(train_loader):
        pca_input = input.numpy()
        input = []
        for image in pca_input:
            input.append(torch.tensor(pca(image)))
        input = torch.stack(input)

        input = input.cuda()

        target = target.cuda()
        model.zero_grad()

        output = model(input)

        loss = criterion(output, target)

        prec = torch.sum(torch.argmax(output.detach().cpu(), dim=1)
                         == torch.argmax(target.detach().cpu(), dim=1))
        precs = (precs*i+prec.item())/(i+1)
        losses = (losses*i+loss.item())/(i+1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print('Epoch:[{0}][{1}/{2}]\t''Loss{loss:.3f}\t''Prec{prec:.3f}(avg:{precs:.3f})'.format(epoch, i, len(train_loader), loss=losses, prec=prec, precs=precs)
                  )

    losses = 0
    precs=0

    model.eval()

    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            pca_input = input.numpy()
            input = []
            for image in pca_input:
                input.append(torch.tensor(pca(image)))
            input = torch.stack(input)

            input = input.cuda()

            target = target.cuda()

            output = model(input)

            loss = criterion(output, target)

            prec = torch.sum(torch.argmax(output.detach().cpu(), dim=1)
                             == torch.argmax(target.detach().cpu(), dim=1))
            precs = (precs*i+prec.item())/(i+1)
            losses = (losses*i+loss.item())/(i+1)

            if i % 10 == 0:
                print('Test:[{0}/{1}]\t''Loss{loss:.3f}\t''Prec{prec:.3f}(avg:{precs:.3f})'.format(i, len(val_loader), loss=losses, prec=prec, precs=precs)
                      )
    lr_decayer.step(losses, epoch)


In [ ]:
torch.save({'model':model.state_dict()},'model06012345.pth.tar')

In [ ]:
# Reading model

with open('model06012306.pth.tar','rb') as f:
    model.load_state_dict(torch.load(f)['model'])

model.double()

## Evaluating the model

In [ ]:
import torchattacks

atk = torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=4)
# atk.set_training_mode(model_training=True, batchnorm_training=True, dropout_training=True)
model.train()

precs = 0
losses = 0
for epoch in range(32):
    for i, (input, target) in enumerate(train_loader):

        input = input.cuda()
        target = target.cuda()
        atk_images = atk(input, target)

        pca_input = atk_images.detach().cpu().numpy()
        input = []
        for image in pca_input:
            input.append(torch.tensor(pca(image)))
        input = torch.stack(input).cuda()
        with torch.no_grad():
            output = model(input)

            # for j in range(len(input)):
            #     cv2.imwrite('image{0}.png'.format(j),input[j].detach().cpu().permute(1,2,0).numpy()*255)
            #     cv2.imwrite('image{0}atk.png'.format(j),atk_images[j].detach().cpu().permute(1,2,0).numpy()*255)

            # print(torch.argmax(target,dim=1))
            # print(torch.argmax(output,dim=1))
            # break
            loss = criterion(output, target)

            prec = torch.sum(torch.argmax(output.detach().cpu(), dim=1)
                             == torch.argmax(target.detach().cpu(), dim=1))
            precs = (precs*i+prec.item())/(i+1)
            losses = (losses*i+loss.item())/(i+1)
            if i % 10 == 0:
                print('Epoch:[{0}][{1}/{2}]\t''Loss{loss:.3f}\t''Prec{prec:.3f}(avg:{precs:.3f})'.format(epoch, i, len(train_loader), loss=losses, prec=prec, precs=precs)
                      )
